In [1]:
import numpy as np
import pandas as pd
import sys
sys.path
sys.path.append('../app')
import calcul
import importlib
import plotly.express as px

In [31]:
data = {
    'fridge': {
        'presence': True,
        'category': 'F',
        'power_kW': 2,
        'min_h': 6,
        'max_h': 8
    },
    'freezer': {
        'presence': False,
        'category': 'F',
        'power_kW': 2.5,
        'min_h': 6,
        'max_h': 8
    },
    'washing_machine': {
        'presence': True,
        'category': 'A',
        'power_kW': 1.5,
        'min_h': 1,
        'max_h': 4
    },
    'dishwasher': {
        'presence': False,
        'category': 'A',
        'power_kW': 2.5,
        'min_h': 1,
        'max_h': 4
    },
    'induction_stove': {
        'presence': False,
        'category': 'A',
        'power_kW': 3,
        'min_h': 1,
        'max_h': 4
    },
    'tv': {
        'presence': True,
        'category': 'L',
        'power_kW': .5,
        'min_h': 4,
        'max_h': 24
    },
    'small_light': {
        'presence': True,
        'category': 'L',
        'power_kW': .1,
        'min_h': 4,
        'max_h': 24
    },
    'big_light': {
        'presence': False,
        'category': 'L',
        'power_kW': .8,
        'min_h': 4,
        'max_h': 24
    },
}

In [64]:
df = calcul.build_base_df(data)

print(f"""Regarding the appliance list, your consumption is expected to be at least \
{df['min_kWh'].sum()} kWh and at most {df['max_kWh'].sum()} kWh""")

df

Regarding the appliance list, your consumption is expected to be at least 15.9 kWh and at most 36.4 kWh


,presence,category,power_kW,min_h,max_h,min_kWh,avg_kWh,max_kWh,ratio_to_avg,ratio_to_max,ratio_to_min
fridge,True,F,2,6,8,12,14.0,16,0.535373,0.43956,0.754717
freezer,False,F,2.5,6,8,0.0,0.0,0.0,0.0,0.0,0.0
washing_machine,True,A,1.5,1,4,1.5,3.75,6.0,0.143403,0.164835,0.09434
dishwasher,False,A,2.5,1,4,0.0,0.0,0.0,0.0,0.0,0.0
induction_stove,False,A,3,1,4,0,0.0,0,0.0,0.0,0.0
tv,True,L,0.5,4,24,2.0,7.0,12.0,0.267686,0.32967,0.125786
small_light,True,L,0.1,4,24,0.4,1.4,2.4,0.053537,0.065934,0.025157
big_light,False,L,0.8,4,24,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
calcul.bargraph(df, 'avg_kWh', 'Average consumption')

min:       15.9 kWh
avg:       26.15 kWh
max:       36.4 kWh
input:     no input yet kWh
        


## Measured consumption input

In [66]:
e_tot = 30

## Frist estimation
- First estimation of the consumion based on the average ratios

In [67]:
df = calcul.first_estimation(df, e_tot)
df[['power_kW', 'min_h', 'max_h',
       'min_kWh', 'avg_kWh', 'max_kWh',
       'esti_kWh', 'esti_h']]

,power_kW,min_h,max_h,min_kWh,avg_kWh,max_kWh,esti_kWh,esti_h
fridge,2.0,6,8,12.0,14.00,16.0,16.000000,8.000000
freezer,2.5,6,8,0.0,0.00,0.0,0.000000,0.000000
washing_machine,1.5,1,4,1.5,3.75,6.0,4.302103,2.868069
dishwasher,2.5,1,4,0.0,0.00,0.0,0.000000,0.000000
induction_stove,3.0,1,4,0.0,0.00,0.0,0.000000,0.000000
tv,0.5,4,24,2.0,7.00,12.0,8.030593,16.061185
small_light,0.1,4,24,0.4,1.40,2.4,1.606119,16.061185
big_light,0.8,4,24,0.0,0.00,0.0,0.000000,0.000000


In [68]:
calcul.bargraph(df, e_tot=e_tot)

min:       15.9 kWh
avg:       26.15 kWh
max:       36.4 kWh
estimated: 29.938814531548758 kWh
input:     30 kWh
        


## Estimation Correction  :
- Problem : due to the high consumption of appliance of type 'F', the total estimation can be above the measured input consumption.
- Solution, concerned appliance are isolated and new estimation is calculated on the other appliances
- The problem can happen a 2nd time with type A appliances

In [70]:
if not calcul.test_overestimate(df):
    print('Overestimation 1')
    df_1, df_2, e_tot_residual = calcul.overestimation_correction(df, e_tot)
    calcul.bargraph(df_2, e_tot=e_tot_residual).show()
    if not calcul.test_overestimate(df_2):
        print('Overestimation 2')
        df_21, df_22, e_tot_residual = calcul.overestimation_correction(df_2, e_tot_residual)
        calcul.bargraph(df_22, e_tot=e_tot_residual).show()
        df = pd.concat([df_1, df_21, df_22])
    else:
        df = pd.concat([df_1, df_2])

if not calcul.test_underestimate(df):
    print('Underestimation 1')
    df_1, df_2, e_tot_residual = calcul.underestimation_correction(df, e_tot)
    calcul.bargraph(df_2, e_tot=e_tot_residual).show()
    if not calcul.test_underestimate(df_2):
        print('Underestimation 2')
        df_21, df_22, e_tot_residual = calcul.underestimation_correction(df_2, e_tot_residual)
        calcul.bargraph(df_22, e_tot=e_tot_residual).show()
        df = pd.concat([df_1, df_21, df_22])
    else:
        df = pd.concat([df_1, df_2])

Underestimation 1
min:       3.9 kWh
avg:       12.15 kWh
max:       20.4 kWh
estimated: 14.0 kWh
input:     14.0 kWh
        


In [71]:
df[['power_kW', 'min_h', 'max_h',
       'min_kWh', 'avg_kWh', 'max_kWh',
       'esti_kWh', 'esti_h']]

,power_kW,min_h,max_h,min_kWh,avg_kWh,max_kWh,esti_kWh,esti_h
fridge,2.0,6,8,12.0,14.00,16.0,16.000000,8.000000
freezer,2.5,6,8,0.0,0.00,0.0,0.000000,0.000000
dishwasher,2.5,1,4,0.0,0.00,0.0,0.000000,0.000000
induction_stove,3.0,1,4,0.0,0.00,0.0,0.000000,0.000000
big_light,0.8,4,24,0.0,0.00,0.0,0.000000,0.000000
washing_machine,1.5,1,4,1.5,3.75,6.0,4.320988,2.868069
tv,0.5,4,24,2.0,7.00,12.0,8.065844,16.061185
small_light,0.1,4,24,0.4,1.40,2.4,1.613169,16.061185


In [72]:
calcul.bargraph(df, e_tot=e_tot)

min:       15.9 kWh
avg:       26.15 kWh
max:       36.4 kWh
estimated: 30.0 kWh
input:     30 kWh
        


## Second correction
- If the measured consumption is really high, close to the maximum (resp.minimum), then estimations are calculated base on the maximum (resp. minimum) ratios instead of the averaged ratios.
ex : 

{
    'fridge': {
        'presence': True,
    },
    'freezer': {
        'presence': False,
    },
    'washing_machine': {
        'presence': True,
    },
    'dishwasher': {
        'presence': False,
    },
    'induction_stove': {
        'presence': False,
    },
    'tv': {
        'presence': True,
    },
    'small_light': {
        'presence': True,
    },
    'big_light': {
        'presence': False,
    },
}
e_tot = 36

In [50]:
df = calcul.estimation_correction_type_2(df, e_tot)

Estimation correct


In [51]:
df[['power_kW', 'min_h', 'max_h',
       'min_kWh', 'avg_kWh', 'max_kWh',
       'esti_kWh', 'esti_h']]

,power_kW,min_h,max_h,min_kWh,avg_kWh,max_kWh,esti_kWh,esti_h
fridge,2.0,6,8,12.0,14.00,16.0,16.000000,8.000000
freezer,2.5,6,8,0.0,0.00,0.0,0.000000,0.000000
dishwasher,2.5,1,4,0.0,0.00,0.0,0.000000,0.000000
induction_stove,3.0,1,4,0.0,0.00,0.0,0.000000,0.000000
big_light,0.8,4,24,0.0,0.00,0.0,0.000000,0.000000
washing_machine,1.5,1,4,1.5,3.75,6.0,6.000000,3.441683
tv,0.5,4,24,2.0,7.00,12.0,11.666667,19.273423
small_light,0.1,4,24,0.4,1.40,2.4,2.333333,19.273423


In [253]:
bargraph(df, e_tot=e_tot)

min:       19.1 kWh
avg:       37.35 kWh
max:       55.6 kWh
estimated: 19.0 kWh
input:     19 kWh
        


In [155]:
fig = px.bar(df, x=df.index, y='esti_kWh')
fig.show()